# amazon web scraper for customer reviews

## amazon search query

Amazon has a specific format for search query within url


1. https://www.amazon.com/s?k=iphone+13
"s?k=" is the path and any string after is the search item

2. https://www.amazon.in/s?k=iphone+13&page=3
same search query as 1. added "&page={page-number}" to scrape mltiple pages

***3 could be wrong or unnecessary***
3. https://www.amazon.com/Web-Scraping-Python-Collecting-Modern/product-reviews/1491910291?pageNumber=2
amazon.../product-name/product-reivews/id

amazon... and product-reviews is fixed
***you must know the id for second url***
id aka ASIN (amazon specific product id, displays each page)


In [32]:
import requests
import operator

from bs4 import BeautifulSoup
from functools import reduce

In [33]:
# global variables
BASE_URL = "https://www.amazon.com/s?k="
HEADERS = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0,5'}
# fake_headers = {"abc": "def"}

In [34]:
# test query
search_query = "iphone+13"
url = BASE_URL + search_query

# include headers to access web
# HEADERS = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
#             'Accept-Language': 'en-US, en;q=0,5'}

search_result = requests.get(url, headers = HEADERS)
# search_result = requests.get(url, headers = fake_headers)


# construct soup
soup = BeautifulSoup(search_result.text, 'html.parser')

# url
# if successful status_code shoud be 200
# search_result.status_code

# raw string of page source code
# search_result.text
# search_result.content

# soup

In [35]:
# user defined func
# HEADERS = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
#             'Accept-Language': 'en-US, en;q=0,5'}

# follow the amazon specific query format, returns string

# scrape the raw data, returns string of the html source code, unformatted, helper func
def getpage(query):
    url = BASE_URL + query
    response = requests.get(url, headers = HEADERS)
    
    if response.status_code == 200:
        return response.text
    else:
        return "Error: status_code != 200"

# convert to html readable string, returns BeautifulSoup object, helper func
def html_code(query):
    return BeautifulSoup(getpage(query), 'html.parser')

# test
search_query = "iphone+13"

# getpage(search_query)
# html_code(search_query)

## inspect the raw html data

1. find product name

notice that there is product name within a span tag with class="a-size-medium a-color-base a-text-normal" 

e.g. <span class="a-size-medium a-color-base a-text-normal">Apple iPhone 13 (128GB, Pink) [Locked] + Carrier Subscription</span>



In [36]:
### use defined function

def getproduct(query, start = 1, end = None):
    """
    query: string input. search items on amazon in browser url tab. 
    if you wanted to search for "iphone 13", it would be "iphone+13"

    start: integer input. starting page number, default = 1
    
    end: integer input. ending page number, default = None

    returns a list of product names from search
    """
    products = []
    
    if end == None:
        query = query + f"&page={str(start)}"
        htmldata = html_code(query)

        products = htmldata.find_all("span", class_ = "a-size-medium a-color-base a-text-normal")
        return list(map(lambda x: x.text, products))

    else:
        query = list(map(lambda x: query + f"&page={str(x)}", range(start, end + 1)))

        for q in query:
            htmldata = html_code(q)
            html_list = htmldata.find_all("span", class_ = "a-size-medium a-color-base a-text-normal")
            products.append(list(map(lambda x: x.text, html_list)))
        
        products = reduce(operator.iconcat, products)
        
        return products
        

# test
search_query = "iphone+13"


# getproduct(search_query)

getproduct(search_query, 1, 5)

['Apple iPhone 13 (128GB, Pink) [Locked] + Carrier Subscription',
 'Apple iPhone 13 Mini (128GB, Starlight) [Locked] + Carrier Subscription',
 'Apple iPhone 13, 128GB, Blue - Unlocked (Renewed)',
 'iPhone 13, 128GB, Blue - Unlocked (Renewed Premium)',
 'iPhone 13 Pro, 128GB, Sierra Blue - Unlocked (Renewed Premium)',
 'Apple iPhone 13 Mini, 128GB, Pink - Unlocked (Renewed)',
 'iPhone 13 Mini, 128GB, Pink - Unlocked (Renewed Premium)',
 'Apple iPhone 13 (128GB, Pink) [Locked] + Carrier Subscription',
 'iPhone 13 Pro Max, 128GB, Graphite - Unlocked (Renewed Premium)',
 'Apple iPhone 12 Pro, 128GB, Graphite - Fully Unlocked (Renewed)',
 'Apple iPhone 12, 128GB, Green - Fully Unlocked (Renewed)',
 'Apple iPhone 11 Pro, US Version, 256GB, Space Gray - Unlocked (Renewed)',
 'Apple iPhone 13 Pro Max, 128GB, Graphite - Unlocked (Renewed)',
 'Apple iPhone 13 Pro, 128GB, Graphite - Unlocked (Renewed)',
 'Apple iPhone 11, 64GB, Yellow - Fully Unlocked (Renewed)',
 'Apple iPhone 12 Pro Max, 128GB,

Object `add` not found.


references

1. https://github.com/vaisakhnambiar/Web-scraping/blob/master/Amazon%20Scraping%20assignment.ipynb
2. https://www.geeksforgeeks.org/web-scraping-amazon-customer-reviews/